<a href="https://colab.research.google.com/github/RogerioMatos75/colab_colmap/blob/main/colab/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
    <picture>
    <source media="(prefers-color-scheme: dark)" srcset="https://docs.nerf.studio/_images/logo-dark.png">
    <source media="(prefers-color-scheme: light)" srcset="https://docs.nerf.studio/_images/logo.png">
    <img alt="nerfstudio" src="https://docs.nerf.studio/_images/logo.png" width="400">
    </picture>
</p>


# Nerfstudio: A collaboration friendly studio for NeRFs


![GitHub stars](https://img.shields.io/github/stars/nerfstudio-project/nerfstudio?color=gold&style=social)

This colab shows how to train and view NeRFs from Nerfstudio both on pre-made datasets or from your own videos/images.

\\

Credit to [NeX](https://nex-mpi.github.io/) for Google Colab format.

## Frequently Asked Questions

*  **Downloading custom data is stalling (no output):**
    * This is a bug in Colab. The data is processing, but may take a while to complete. You will know processing completed if `data/nerfstudio/custom_data/transforms.json` exists. Terminating the cell early will result in not being able to train.
*  **Processing custom data is taking a long time:**
    * The time it takes to process data depends on the number of images and its resolution. If processing is taking too long, try lowering the resolution of your custom data.
*  **Error: Data processing did not complete:**
    * This means that the data processing script did not fully complete. This could be because there were not enough images, or that the images were of low quality. We recommend images with little to no motion blur and lots of visual overlap of the scene to increase the chances of successful processing.
*   **Training is not showing progress**:
    * The lack of output is a bug in Colab. You can see the training progress from the viewer.
* **Viewer Quality is bad / Low resolution**:
    * This may be because more GPU is being used on training that rendering the viewer. Try pausing training or decreasing training utilization.
* **WARNING: Running pip as the 'root' user...:**:
    * This and other pip warnings or errors can be safely ignored.
* **Other problems?**
    * Feel free to create an issue on our [GitHub repo](https://github.com/nerfstudio-project/nerfstudio).


In [18]:
t# @markdown <h1>Install Nerfstudio and Dependencies (~8 min)</h1>

%cd /content/
!pip install --upgrade pip
# Use compatible torch and torchvision versions for Python 3.10 and CUDA 11.8
!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchaudio==0.13.1+cu117 --extra-index-url https://download.pytorch.org/whl/cu117

# Installing TinyCuda
%cd /content/
# Use compatible tinycudann version for Python 3.10 and CUDA 11.8
!gdown "https://drive.google.com/u/1/uc?id=1EEfC83K6t4e6lX36t6k-fF2X4pT3b9eM&confirm=t"
!pip install tinycudann-1.6-cp310-cp310-linux_x86_64.whl

# Installing COLMAP
%cd /content/
!apt-get install colmap

# Install nerfstudio
%cd /content/
!pip install git+https://github.com/nerfstudio-project/nerfstudio.gi

/content
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu117
ERROR: Could not find a version that satisfies the requirement torch==1.13.1+cu117 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0, 2.7.0, 2.7.1, 2.8.0)
ERROR: No matching distribution found for torch==1.13.1+cu117
/content
Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/u/1/uc?id=1EEfC83K6t4e6lX36t6k-fF2X4pT3b9eM&confirm=t

but Gdown can't. Please check connections and permissions.
ERROR: tinycudann-1.6-cp310-cp310-linux_x86_64.whl is not a supported wheel on this platform.
/content
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
colm

In [30]:
# @markdown <h1> Downloading and Processing Data</h1>
# @markdown <h3>Pick the preset scene or upload your own images/video</h3>
import glob
import os

from google.colab import files
from IPython.core.display import HTML, display

scene = "📤 upload your images"  # @param ['🖼 poster', '🚜 dozer', '🌄 desolation', '📤 upload your images' , '🎥 upload your own video', '🔺 upload Polycam data', '💽 upload your own Record3D data']
scene = " ".join(scene.split(" ")[1:])

if scene == "upload Polycam data":
    %cd /content/
    !mkdir -p /content/data/nerfstudio/custom_data
    %cd /content/data/nerfstudio/custom_data/
    uploaded = files.upload()
    dir = os.getcwd()
    if len(uploaded.keys()) > 1:
        print("ERROR, upload a single .zip file when processing Polycam data")
    dataset_dir = [os.path.join(dir, f) for f in uploaded.keys()][0]
    !ns-process-data polycam --data $dataset_dir --output-dir /content/data/nerfstudio/custom_data/
    scene = "custom_data"
elif scene == "upload your own Record3D data":
    display(HTML("<h3>Zip your Record3D folder, and upload.</h3>"))
    display(
        HTML(
            '<h3>More information on Record3D can be found <a href="https://docs.nerf.studio/en/latest/quickstart/custom_dataset.html#record3d-capture" target="_blank">here</a>.</h3>'
        )
    )
    %cd /content/
    !mkdir -p /content/data/nerfstudio/custom_data
    %cd /content/data/nerfstudio/custom_data/
    uploaded = files.upload()
    dir = os.getcwd()
    preupload_datasets = [os.path.join(dir, f) for f in uploaded.keys()]
    record_3d_zipfile = preupload_datasets[0]
    !unzip $record_3d_zipfile -d /content/data/nerfstudio/custom_data
    custom_data_directory = glob.glob("/content/data/nerfstudio/custom_data/*")[0]
    !ns-process-data record3d --data $custom_data_directory --output-dir /content/data/nerfstudio/custom_data/
    scene = "custom_data"
elif scene in ["upload your images", "upload your own video"]:
    display(HTML("<h3>Select your custom data</h3>"))
    display(HTML("<p/>You can select multiple images by pressing ctrl, cmd or shift and click.<p>"))
    display(
        HTML(
            "<p/>Note: This may take time, especially on higher resolution inputs, so we recommend to download dataset after creation.<p>"
        )
    )
    !mkdir -p /content/data/nerfstudio/custom_data
    if scene == "upload your images":
        !mkdir -p /content/data/nerfstudio/custom_data/raw_images
        %cd /content/data/nerfstudio/custom_data/raw_images
        uploaded = files.upload()
        dir = os.getcwd()
    else:
        %cd /content/data/nerfstudio/custom_data/
        uploaded = files.upload()
        dir = os.getcwd()
    preupload_datasets = [os.path.join(dir, f) for f in uploaded.keys()]
    del uploaded
    %cd /content/

    if scene == "upload your images":
        !ns-process-data images --data /content/data/nerfstudio/custom_data/raw_images --output-dir /content/data/nerfstudio/custom_data/
    else:
        video_path = preupload_datasets[0]
        !ns-process-data video --data $video_path --output-dir /content/data/nerfstudio/custom_data/

    scene = "custom_data"
else:
    %cd /content/
    !ns-download-data nerfstudio --capture-name=$scene

print("Data Processing Succeeded!")

/content/data/nerfstudio/custom_data


Saving frame_0001.png to frame_0001.png
Saving frame_0002.png to frame_0002.png
Saving frame_0003.png to frame_0003.png
Saving frame_0004.png to frame_0004.png
Saving frame_0005.png to frame_0005.png
Saving frame_0006.png to frame_0006.png
Saving frame_0007.png to frame_0007.png
Saving frame_0008.png to frame_0008.png
Saving frame_0009.png to frame_0009.png
Saving frame_0010.png to frame_0010.png
Saving frame_0011.png to frame_0011.png
Saving frame_0012.png to frame_0012.png
Saving frame_0013.png to frame_0013.png
Saving frame_0014.png to frame_0014.png
Saving frame_0015.png to frame_0015.png
Saving frame_0016.png to frame_0016.png
Saving frame_0017.png to frame_0017.png
Saving frame_0018.png to frame_0018.png
Saving frame_0019.png to frame_0019.png
Saving frame_0020.png to frame_0020.png
Saving frame_0021.png to frame_0021.png
Saving frame_0022.png to frame_0022.png
Saving frame_0023.png to frame_0023.png
Saving frame_0024.png to frame_0024.png
Saving frame_0025.png to frame_0025.png


In [24]:
# @markdown <h1>Start Training</h1>

%cd /content
!pip install colab-xterm
%load_ext colabxterm
%env TERM=xterm
from IPython.display import clear_output

clear_output(wait=True)
if os.path.exists(f"data/nerfstudio/{scene}/transforms.json"):
    print(
        "\033[1m"
        + "Copy and paste the following command into the terminal window that pops up under this cell."
        + "\033[0m"
    )
    print(
        f"ns-train nerfacto --viewer.websocket-port 7007 --viewer.make-share-url True nerfstudio-data --data data/nerfstudio/{scene} --downscale-factor 4"
    )
    print()
    %xterm
else:
    from IPython.core.display import HTML, display

    display(HTML('<h3 style="color:red">Error: Data processing did not complete</h3>'))
    display(HTML("<h3>Please re-run `Downloading and Processing Data`, or view the FAQ for more info.</h3>"))

In [31]:
# @title # Render Video { vertical-output: true }
# @markdown <h3>Export the camera path from within the viewer, then run this cell.</h3>
# @markdown <h5>The rendered video should be at renders/output.mp4!</h5>


base_dir = "/content/outputs/unnamed/nerfacto/"
training_run_dir = base_dir + os.listdir(base_dir)[0]

from IPython.core.display import HTML, display

display(HTML("<h3>Upload the camera path JSON.</h3>"))
%cd $training_run_dir
uploaded = files.upload()
uploaded_camera_path_filename = list(uploaded.keys())[0]

config_filename = training_run_dir + "/config.yml"
camera_path_filename = training_run_dir + "/" + uploaded_camera_path_filename
camera_path_filename = camera_path_filename.replace(" ", "\\ ").replace("(", "\\(").replace(")", "\\)")

%cd /content/
!ns-render camera-path --load-config $config_filename --camera-path-filename $camera_path_filename --output-path renders/output.mp4

FileNotFoundError: [Errno 2] No such file or directory: '/content/outputs/unnamed/nerfacto/'

In [32]:
# Cria a pasta para guardar os frames
!mkdir frames_final

# Executa o FFmpeg para extrair os frames do vídeo para a nova pasta
!ffmpeg -i Photorealistic_Character_Video_Generation.mp4 -qscale:v 2 frames_final/frame_%04d.png


mkdir: cannot create directory ‘frames_final’: File exists
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --

In [33]:
print("---===[ ETAPA 1 de 4: Extraindo pontos-chave (Modo CPU Forçado) ]===---")
!colmap feature_extractor \
    --database_path database.db \
    --image_path frames_final \
    --SiftExtraction.use_gpu 0  # <--- A MÁGICA QUE RESOLVE O PROBLEMA

print("\n---===[ ETAPA 2 de 4: Combinando pontos-chave ]===---")
!colmap exhaustive_matcher \
    --database_path database.db

print("\n---===[ ETAPA 3 de 4: Mapeando a cena ]===---")
!mkdir -p sparse
!colmap mapper \
    --database_path database.db \
    --image_path frames_final \
    --output_path sparse

print("\n---===[ ETAPA 4 de 4: Removendo distorção das imagens ]===---")
!mkdir -p dense
!colmap image_undistorter \
    --image_path frames_final \
    --input_path sparse/0 \
    --output_path dense

print("\n---===[ PROCESSAMENTO COLMAP CONCLUÍDO! ]===---")

---===[ ETAPA 1 de 4: Extraindo pontos-chave (Modo CPU Forçado) ]===---

Feature extraction

Elapsed time: 0.000 [minutes]

---===[ ETAPA 2 de 4: Combinando pontos-chave ]===---
qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: eglfs, linuxfb, minimal, minimalegl, offscreen, vnc, xcb.

*** Aborted at 1758739844 (unix time) try "date -d @1758739844" if you are using GNU date ***
PC: @                0x0 (unknown)
*** SIGABRT (@0x960a) received by PID 38410 (TID 0x7e5dc41d1080) from PID 38410; stack trace: ***
    @     0x7e5dcc787046 (unknown)
    @     0x7e5dca808520 (unknown)
    @     0x7e5dca85c9fc pthread_kill
    @     0x7e5dca808476 raise
    @     0x7e5dca7ee7f3 abort
    @     0x7e5dcad92ba3 QMessageLogger::fatal()
    @     0x7e5